In [ ]:
import os
import re
import warnings
import joblib
from pathlib import Path
from math import log1p, ceil
import datetime
import io # For reading string CSVs

from dotenv import load_dotenv
import polars as pl
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer # For handling any NaNs before scaling
from sklearn.ensemble import IsolationForest

import sqlalchemy as sa

import numpy as np

# For reproducible results
np.random.seed(42)

# Polars settings
pl.Config.set_tbl_rows(30) # Show more rows
pl.Config.set_tbl_cols(80) # Show more columns by default

# --- Database Connection and Query (from your code) ---
load_dotenv()
engine = sa.create_engine(os.environ["WAREHOUSE_COOLIFY_URL"], pool_pre_ping=True)

import time

DAYS_BACK = 28
# seconds‑since‑epoch threshold in Python (no DB date math)
cutoff = int(time.time() - DAYS_BACK * 86_400)

query = f"""
SELECT
    /* convert only the rows that survive the WHERE filter */
    CASE
        WHEN time > 32503680000*1000 THEN time/1000000               -- μs → s
        WHEN time > 32503680000      THEN time/1000                  -- ms → s
        ELSE                           time                          -- s
    END::double precision                AS ts_sec,

    *
FROM hackatime.heartbeats
/* pre‑filter in native units so the index on `time` is usable */
WHERE (
        (time <= 32503680000                  AND time >= {cutoff})           -- seconds
     OR (time >  32503680000  AND time <= 32503680000*1000
                                         AND time >= {cutoff*1000})          -- milliseconds
     OR (time >  32503680000*1000          AND time >= {cutoff*1000000})      -- microseconds
)
AND category = 'coding'
AND entity NOT IN ('test.txt', 'welcome.txt')
ORDER BY heartbeats.user_id, heartbeats.time;
"""

try:
    with engine.begin() as conn:
        heartbeats = pl.read_database(query, connection=conn, infer_schema_length=None)
    # already ordered correctly by query, but keep for safety
    heartbeats = heartbeats.sort(["user_id", "ts_sec"])
    print(f"Successfully loaded {heartbeats.shape[0]} heartbeats from the database.")
except Exception as e:
    print(f"Could not load data from database: {e}")
    print("Proceeding with empty or sample data (if defined below).")

heartbeats.head()

In [ ]:
# Known good/bad user IDs for validation (replace with your actual lists)
KNOWN_GOOD_USER_IDS = {
    1, # max
    2, # zrl
    104, # acon
    69, # malted
    864, # thomas
    664, # lux
    10, # annabel
    41, # neon
}
KNOWN_BAD_USER_IDS = {
    1613,
    1728,
    18,
    1688
}

In [ ]:
# Previous part of Step 1 remains the same...

if heartbeats is not None and not heartbeats.is_empty():
    # Basic type conversions and fill nulls
    numerical_cols_to_fill_zero = ["line_additions", "line_deletions", "lineno", "lines", "cursorpos", "project_root_count"]
    fill_zero_exprs = [pl.col(c).fill_null(0) for c in numerical_cols_to_fill_zero if c in heartbeats.columns]

    categorical_cols_to_fill_unknown = ["branch", "language", "machine", "project", "editor"] # `editor` might be from user_agent
    fill_unknown_exprs = [pl.col(c).fill_null("Unknown") for c in categorical_cols_to_fill_unknown if c in heartbeats.columns]

    heartbeats_processed = heartbeats.with_columns(
        fill_zero_exprs +
        fill_unknown_exprs +
        [
            pl.col("is_write").cast(pl.Boolean).fill_null(False),
            pl.col("ts_sec").cast(pl.Float64),
            pl.from_epoch(pl.col("ts_sec").round(0).cast(pl.Int64), time_unit="s").alias("datetime_utc"), # Rounded for from_epoch
        ]
    )

    # User Agent Parsing (Simplified example - you might need a more robust parser)
    # This extracts a potential plugin/editor name like 'vscode-wakatime', 'cursor', etc.
    def extract_plugin_name_simple(user_agent_str: str | None) -> str: # Added type hint for input
        if user_agent_str is None: return "Unknown"
        # Example: wakatime/v1.115.2 (darwin-23.3.0-arm64) go1.24.2 cursor/1.96.2 vscode-wakatime/25.0.1
        # Look for patterns like editor/version or plugin-wakatime/version
        # This regex tries to find something like 'vscode-wakatime' or 'cursor'
        
        # Try to find specific known patterns first
        patterns = [
            r"vscode-wakatime(?:/[\d\.]+)?", # vscode-wakatime/25.0.1 or vscode-wakatime
            r"cursor(?:/[\d\.]+)?",          # cursor/1.96.2 or cursor
            r"jetbrains(?:-\w+)?(?:/[\d\.]+)?", # jetbrains-phpstorm/2023.1 or jetbrains/2023.1
            r"wakatime-cli(?:/[\w\.\-]+)?",  # wakatime-cli/0.0.0-dev.9+git.732ce039
            r"vim(?:/[\d\.]+)?",
            r"neovim(?:/[\d\.]+)?",
            r"sublime(?:/[\d\.]+)?",
            r"atom(?:/[\d\.]+)?",
            r"jupyter(?:/[\d\.]+)?",
            r"wakatime(?:/[\w\.\-]+)?" # General wakatime prefix
        ]
        
        for p_str in patterns:
            match = re.search(p_str, user_agent_str, re.IGNORECASE)
            if match:
                name = match.group(0).lower()
                if 'vscode-wakatime' in name or 'vscode' in name: return 'vscode'
                if 'cursor' in name: return 'cursor'
                if 'jetbrains' in name: return 'jetbrains'
                if 'neovim' in name: return 'neovim'
                if 'vim' in name: return 'vim'
                if 'sublime' in name: return 'sublime'
                if 'atom' in name: return 'atom'
                if 'jupyter' in name: return 'jupyter'
                if 'wakatime-cli' in name: return 'wakatime-cli'
                if 'wakatime' in name: return 'wakatime-generic' # a fallback if no specific editor named
                return name.split('/')[0].split('-')[0] # Basic cleaning

        # Fallback to the first part if it looks like an agent name
        first_part_match = re.match(r"([a-zA-Z0-9_-]+)/", user_agent_str)
        if first_part_match:
            return first_part_match.group(1)
        return "Unknown"

    heartbeats_processed = heartbeats_processed.with_columns(
        # CORRECTED LINE:
        pl.col("user_agent").map_elements(extract_plugin_name_simple, return_dtype=pl.Utf8).alias("plugin_name")
    )
    
    print("Data cleaning and preparation complete.")
    print(heartbeats_processed.head())
    print("\nValue counts for `plugin_name`:")
    print(heartbeats_processed['plugin_name'].value_counts())

else:
    print("Skipping Step 1 as heartbeats data is not loaded.")
    heartbeats_processed = pl.DataFrame() # Empty dataframe to prevent errors later

In [ ]:
HEARTBEAT_TIMEOUT = 120.0  # seconds

if not heartbeats_processed.is_empty():
    # Calculate time diff to next heartbeat within each user's activity
    # Ensure sorting by user_id and ts_sec first
    heartbeats_processed = heartbeats_processed.sort(["user_id", "ts_sec"])
    
    heartbeats_timed = heartbeats_processed.with_columns(
        (pl.col("ts_sec").shift(-1).over("user_id") - pl.col("ts_sec")).alias("time_to_next_hb")
    )

    # Calculate duration for each heartbeat
    heartbeats_timed = heartbeats_timed.with_columns(
        pl.when(pl.col("time_to_next_hb").is_null())
        .then(pl.lit(HEARTBEAT_TIMEOUT))
        .when(pl.col("time_to_next_hb") > HEARTBEAT_TIMEOUT)
        .then(pl.lit(HEARTBEAT_TIMEOUT))
        .otherwise(pl.col("time_to_next_hb"))
        .alias("heartbeat_coded_duration_sec")
    )

    # Define sessions
    heartbeats_sessioned = heartbeats_timed.with_columns(
        (pl.col("ts_sec") - pl.col("ts_sec").shift(1).over("user_id")).alias("time_from_prev_hb")
    )

    heartbeats_sessioned = heartbeats_sessioned.with_columns(
        pl.when(pl.col("time_from_prev_hb").is_null() | (pl.col("time_from_prev_hb") > HEARTBEAT_TIMEOUT))
        .then(True)
        .otherwise(False)
        .alias("is_session_start")
    )

    heartbeats_sessioned = heartbeats_sessioned.with_columns(
        pl.col("is_session_start").cum_sum().over("user_id").alias("session_id_within_user")
    )
    heartbeats_sessioned = heartbeats_sessioned.with_columns(
        (pl.col("user_id").cast(pl.Utf8) + "_s" + pl.col("session_id_within_user").cast(pl.Utf8)).alias("global_session_id")
    )
    
    print("Coding durations and sessions calculated.")
    print(heartbeats_sessioned.select([
        "user_id", "ts_sec", "time_to_next_hb", "heartbeat_coded_duration_sec", 
        "time_from_prev_hb", "is_session_start", "global_session_id"
    ]).head(10))
else:
    print("Skipping Step 2 as heartbeats_processed is empty.")
    heartbeats_sessioned = pl.DataFrame()

In [ ]:
if not heartbeats_sessioned.is_empty():
    user_features_list = []

    # --- 1. Total Coding Time & Basic Counts ---
    user_total_time_and_counts = heartbeats_sessioned.group_by("user_id").agg(
        pl.sum("heartbeat_coded_duration_sec").alias("total_coded_seconds"),
        pl.len().alias("total_heartbeats"), # CORRECTED: pl.count() -> pl.len() for row count in group_by
        # CORRECTED: Chain .sum() onto the expression
        pl.col("is_write").cast(pl.UInt8).sum().alias("num_write_heartbeats") 
    ).with_columns(
        (pl.col("num_write_heartbeats") / pl.col("total_heartbeats")).fill_null(0).alias("ratio_write_heartbeats"),
        (pl.col("total_heartbeats") / (pl.col("total_coded_seconds") / 3600.0 + 1e-6)) # Add epsilon to avoid div by zero
            .fill_null(0).alias("heartbeats_per_coding_hour")
    )
    user_features_list.append(user_total_time_and_counts.drop("num_write_heartbeats")) # Drop intermediate

    # --- 2. Session-based features ---
    # (Rest of the code for session aggregates)
    session_aggregates = heartbeats_sessioned.group_by("user_id", "global_session_id").agg(
        pl.sum("heartbeat_coded_duration_sec").alias("session_duration_sec"),
        pl.len().alias("num_heartbeats_in_session"), # CORRECTED: pl.count() -> pl.len()
        pl.min("ts_sec").alias("session_start_ts"),
        pl.max("ts_sec").alias("session_end_ts")
    )
    
    user_session_stats = session_aggregates.group_by("user_id").agg(
        pl.len().alias("num_sessions"), # If counting session_ids per user
        pl.mean("session_duration_sec").alias("avg_session_duration_sec"),
        pl.max("session_duration_sec").alias("max_session_duration_sec"),
        pl.median("session_duration_sec").alias("median_session_duration_sec"),
        pl.std("session_duration_sec").fill_null(0).alias("std_session_duration_sec"),
        pl.mean("num_heartbeats_in_session").alias("avg_heartbeats_per_session"),
        (pl.col("session_duration_sec").filter(pl.col("session_duration_sec") > 4 * 3600).count()).alias("num_long_sessions_gt_4h") # Alternative way to count conditional
        # Or using when/then/otherwise/sum for conditional count:
        # (pl.when(pl.col("session_duration_sec") > 4 * 3600).then(1).otherwise(0)).sum().alias("num_long_sessions_gt_4h")
    )
    user_features_list.append(user_session_stats)

    # --- 3. Activity Regularity ---
    user_activity_regularity = heartbeats_sessioned.group_by("user_id").agg(
        pl.mean("time_from_prev_hb").fill_null(HEARTBEAT_TIMEOUT).alias("avg_time_between_heartbeats"), # Fill null for first hb
        pl.std("time_from_prev_hb").fill_null(0).alias("std_time_between_heartbeats")
    )
    user_features_list.append(user_activity_regularity)

    # --- 4. File Interaction / "No discernible changes" ---
    heartbeats_changes = heartbeats_sessioned.with_columns(
        ((pl.col("entity") == pl.col("entity").shift(1).over("user_id")) & \
         (pl.col("fields_hash") == pl.col("fields_hash").shift(1).over("user_id")) & \
         (pl.col("time_from_prev_hb") <= HEARTBEAT_TIMEOUT) 
        ).alias("is_no_change_streak_hb")
    )
    
    user_no_change_stats = heartbeats_changes.group_by("user_id").agg(
        # CORRECTED: Chain .sum()
        pl.col("is_no_change_streak_hb").cast(pl.UInt8).sum().alias("num_no_change_streak_heartbeats")
    ).join(user_total_time_and_counts.select("user_id", "total_heartbeats"), on="user_id", how="left")
    
    user_no_change_stats = user_no_change_stats.with_columns(
        (pl.col("num_no_change_streak_heartbeats") / pl.col("total_heartbeats")).fill_null(0).alias("ratio_no_change_streak_hbs")
    )
    user_features_list.append(user_no_change_stats.select("user_id", "ratio_no_change_streak_hbs"))

    # Lines changed stats
    user_loc_stats = heartbeats_processed.group_by("user_id").agg(
        pl.sum("line_additions").alias("total_line_additions"),
        pl.sum("line_deletions").alias("total_line_deletions"),
    ).with_columns(
        (pl.col("total_line_additions") + pl.col("total_line_deletions")).alias("total_loc_changed")
    ).join(user_total_time_and_counts.select("user_id", "total_coded_seconds"), on="user_id", how="left")
    
    user_loc_stats = user_loc_stats.with_columns(
        (pl.col("total_loc_changed") / (pl.col("total_coded_seconds") / 3600.0 + 1e-6)) 
            .fill_null(0).alias("loc_changed_per_hour")
    )
    user_features_list.append(user_loc_stats.select(["user_id", "total_loc_changed", "loc_changed_per_hour"]))

    # --- 5. Diversity of Activity ---
    user_diversity_stats = heartbeats_processed.group_by("user_id").agg(
        pl.n_unique("project").alias("num_unique_projects"),
        pl.n_unique("language").alias("num_unique_languages"),
        pl.n_unique("entity").alias("num_unique_files"),
        pl.n_unique("ip_address").alias("num_unique_ips"),
        pl.n_unique("operating_system").alias("num_unique_os"),
        pl.n_unique("plugin_name").alias("num_unique_plugins")
    )
    user_features_list.append(user_diversity_stats)

    # --- 6. Coding Time Patterns ---
    heartbeats_with_hour = heartbeats_processed.with_columns(
        pl.col("datetime_utc").dt.hour().alias("hour_of_day_utc")
    )
    user_time_patterns = heartbeats_with_hour.group_by("user_id").agg(
        # CORRECTED: Chain .sum() for conditional sums
        (pl.when((pl.col("hour_of_day_utc") >= 0) & (pl.col("hour_of_day_utc") < 6)).then(1).otherwise(0)).sum()
            .truediv(pl.len()) # Use truediv for float division
            .fill_null(0).alias("prop_night_coding_utc0_6"),
        (pl.when(pl.col("datetime_utc").dt.weekday().is_in([6,7])).then(1).otherwise(0)).sum()
            .truediv(pl.len())
            .fill_null(0).alias("prop_weekend_coding")
    )
    user_features_list.append(user_time_patterns)

    # --- 7. Data Fullness/Consistency (Simple version) ---
    user_data_fullness = heartbeats.group_by("user_id").agg( 
        # CORRECTED: Chain .sum() for conditional sums
        pl.col("language").is_null().cast(pl.UInt8).sum().truediv(pl.len()).alias("prop_null_language_orig"),
        pl.col("line_additions").is_null().cast(pl.UInt8).sum().truediv(pl.len()).alias("prop_null_line_additions_orig"),
        (pl.col("branch").is_null() | (pl.col("branch") == "")).cast(pl.UInt8).sum().truediv(pl.len()).alias("prop_empty_branch_orig")
    )
    user_features_list.append(user_data_fullness)
    
    # --- Combine all feature DataFrames ---
    if user_features_list:
        df_features = user_features_list[0]
        for i in range(1, len(user_features_list)):
            df_features = df_features.join(user_features_list[i], on="user_id", how="left")

        df_features = df_features.fill_null(0) # Global fillna after all joins
        
        cols_to_log_transform = ["total_coded_seconds", "total_heartbeats", "total_loc_changed", "max_session_duration_sec"]
        for col_name in cols_to_log_transform:
            if col_name in df_features.columns:
                 df_features = df_features.with_columns(
                    pl.col(col_name).log1p().alias(f"{col_name}_log1p")
                 )
        
        print("Feature engineering complete.")
        print(f"Generated {len(df_features.columns) -1} features for {df_features.shape[0]} users.")
        print(df_features.head())
    else:
        print("No features generated, list is empty.")
        df_features = pl.DataFrame()
else:
    print("Skipping Step 3 as heartbeats_sessioned is empty.")
    df_features = pl.DataFrame()

In [ ]:
if not df_features.is_empty() and "user_id" in df_features.columns:
    # Prepare data for sklearn
    user_ids_for_results = df_features["user_id"].to_list() # Keep user_ids for mapping results
    
    # Select only feature columns (all numeric, including log-transformed ones if created)
    # Exclude original versions of log-transformed columns if you prefer
    feature_columns = [
        col for col in df_features.columns 
        if col not in ["user_id"] and ("_log1p" in col or col not in [c.replace("_log1p","") for c in df_features.columns if "_log1p" in c])
    ]
    # Ensure no non-numeric columns slip through
    X_df = df_features.select(feature_columns)
    final_feature_columns = []
    for col_name in X_df.columns:
        if X_df[col_name].dtype in [pl.Float32, pl.Float64, pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64]:
            final_feature_columns.append(col_name)
        else:
            print(f"Warning: Column {col_name} is not numeric ({X_df[col_name].dtype}) and will be excluded from model training.")
            
    X = X_df.select(final_feature_columns).to_numpy()
    print(f"Training model with {X.shape[1]} features: {final_feature_columns}")

    # Sklearn pipeline
    # `contamination` is the expected proportion of outliers. 'auto' is often a good start.
    # You can tune this based on how many anomalies you expect or can investigate.
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')), # Handles any lingering NaNs
        ('scaler', StandardScaler()),
        ('iso_forest', IsolationForest(n_estimators=100,       # Default, can tune
                                       contamination='auto', # Expected proportion of outliers
                                       # contamination=0.05, # Or set a specific value e.g. 5%
                                       random_state=42,
                                       n_jobs=-1))          # Use all available cores
    ])

    # Fit the pipeline
    pipeline.fit(X)

    # Get anomaly scores and predictions
    # decision_function: The lower, the more abnormal. Outliers have lower scores.
    # For IF, scores are not bounded, but typically negative for outliers.
    anomaly_scores = pipeline.decision_function(X) # No need to re-transform X, pipeline handles it
    
    # predict: -1 for outliers (anomalies), 1 for inliers (normal)
    predictions = pipeline.predict(X)

    # Add results back to df_features for analysis
    df_results = df_features.with_columns([
        pl.Series("anomaly_score", anomaly_scores),
        pl.Series("prediction", predictions) # -1 for anomaly, 1 for normal
    ])
    
    print("Model training and prediction complete.")
    print(df_results.select(["user_id", "anomaly_score", "prediction"]).sort("anomaly_score").head())
    print(f"Number of users flagged as anomalies (prediction = -1): {df_results.filter(pl.col('prediction') == -1).shape[0]}")

else:
    print("Skipping Step 4 as df_features is empty or missing 'user_id'.")
    df_results = pl.DataFrame()

In [ ]:
if not df_results.is_empty() and "user_id" in df_results.columns:
    # Add ground_truth_label for evaluation
    df_results = df_results.with_columns(
        pl.when(pl.col("user_id").is_in(KNOWN_GOOD_USER_IDS)).then(0) # 0 for good
        .when(pl.col("user_id").is_in(KNOWN_BAD_USER_IDS)).then(1)   # 1 for bad
        .otherwise(None) # None for users not in known lists
        .alias("ground_truth_label")
    )

    df_results_sorted = df_results.sort("anomaly_score")

    print("\n--- Top 10 Most Anomalous Users ---")
    print(df_results_sorted.select(["user_id", "anomaly_score", "prediction", "ground_truth_label", "total_coded_seconds_log1p", "heartbeats_per_coding_hour", "ratio_no_change_streak_hbs"]).head(10))

    known_bad_flagged = df_results_sorted.filter((pl.col("ground_truth_label") == 1))
    print("\n--- Scores for Known BAD Users ---")
    if not known_bad_flagged.is_empty():
        print(known_bad_flagged.select(["user_id", "anomaly_score", "prediction", "ground_truth_label"]))
        print(f"Total known bad: {len(KNOWN_BAD_USER_IDS)}. Found by model: {known_bad_flagged.filter(pl.col('prediction') == -1).shape[0]}")
    else:
        print("No known bad users in the dataset or results.")
        
    false_positives = df_results_sorted.filter((pl.col("ground_truth_label") == 0) & (pl.col("prediction") == -1))
    print("\n--- Known GOOD Users Flagged as Anomalies (False Positives) ---")
    if not false_positives.is_empty():
        print(false_positives.select(["user_id", "anomaly_score", "prediction", "ground_truth_label"]))
    else:
        print("No known good users were flagged as anomalies.")

    # --- Basic Metrics (if enough known labels) ---
    from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, auc

    # Filter for users with known ground truth
    df_known_users = df_results.filter(pl.col("ground_truth_label").is_not_null())

    if not df_known_users.is_empty() and df_known_users["ground_truth_label"].n_unique() > 1:
        y_true_known = df_known_users["ground_truth_label"].to_numpy()
        # Map IF predictions: 1 (inlier) -> 0 (normal class for metrics), -1 (outlier) -> 1 (anomaly class for metrics)
        y_pred_known_if_mapped = np.array([1 if p == -1 else 0 for p in df_known_users["prediction"].to_numpy()])
        
        # decision_function scores: lower means more anomalous. For ROC AUC, higher scores usually mean positive class.
        # So, we use -anomaly_scores as the score for the "positive" (anomalous) class.
        y_scores_known_if = -df_known_users["anomaly_score"].to_numpy()

        print("\n--- Evaluation Metrics on Known Users ---")
        print(f"Number of known good users: {sum(y_true_known == 0)}")
        print(f"Number of known bad users: {sum(y_true_known == 1)}")

        print("\nClassification Report (IF predictions vs Ground Truth):")
        # target_names=['Good User (Inlier)', 'Bad User (Outlier)']
        print(classification_report(y_true_known, y_pred_known_if_mapped, zero_division=0))

        print("\nConfusion Matrix ([Good, Bad] x [Predicted Good, Predicted Bad]):")
        # Rows: True class (Good, Bad). Cols: Predicted class (Good, Bad)
        # TN  FP
        # FN  TP
        cm = confusion_matrix(y_true_known, y_pred_known_if_mapped)
        print(cm)
        
        try:
            roc_auc = roc_auc_score(y_true_known, y_scores_known_if)
            print(f"\nROC AUC Score (on known users): {roc_auc:.4f}")
            
            precision, recall, _ = precision_recall_curve(y_true_known, y_scores_known_if)
            pr_auc = auc(recall, precision) # Area under PR curve
            print(f"Precision-Recall AUC Score (on known users): {pr_auc:.4f}")

        except ValueError as e_auc:
            print(f"Could not calculate AUC scores (likely only one class present in y_true after filtering): {e_auc}")
            
    else:
        print("\nNot enough known users with diverse labels to calculate detailed metrics.")

    # --- Further Analysis & Iteration Ideas ---
    # 1. Inspect Features of Anomalies: Look at the feature values for users with low anomaly_scores.
    #    Do they align with your definition of suspicious behavior?
    #    `df_results_sorted.head(20)` is a good place to start.
    #
    # 2. Tune `contamination`: If IsolationForest flags too many or too few users, adjust this parameter.
    #    It's the expected proportion of outliers.
    #
    # 3. Feature Refinement:
    #    - Are any features too noisy or not discriminating well?
    #    - Can you create more sophisticated features? E.g.,
    #      - Longest continuous session on a single file with no `fields_hash` changes.
    #      - Deviations from typical behavior for a given `plugin_name` (e.g., if a VSCode user suddenly has no line changes reported for hours).
    #      - Velocity metrics: LOC changed per *actual* minute of file interaction, not just overall session time.
    #
    # 4. Daily Aggregations: For "disproportionate hours day after day", you might need to aggregate features per user per day,
    #    then look for users with many consecutive anomalous days, or high average daily anomaly scores.
    #    This would be a second layer of analysis on top of daily anomaly scores.
    #
    # 5. Model Exploration: If Isolation Forest doesn't give satisfactory results after tuning,
    #    you could explore other unsupervised methods like Local Outlier Factor (LOF) or One-Class SVM,
    #    though Isolation Forest is generally a strong baseline.

else:
    print("Skipping Step 5 as df_results is empty or missing 'user_id'.")